# Problem 1 - Machine Scheduling

## (a)

In [1]:
using Random
seed = 425 # seed for data generation

N = 1:60 # jobs
M = 1:30 # machines

Random.seed!(seed)

# set time lengths of jobs on each machine
a = zeros(length(M),length(N)) 
for i in M
    for j in N
        a[i,j] = round(6+Random.rand()*Random.rand(6:25),digits=2)
    end 
end

# capacity of each machine
b = Dict(zip(M,[12*sum(a[i,j] for j in N)/length(M) for i in M]))

# cost of running jobs on each machine
c = zeros(length(M),length(N)) 
for i in M
    for j in N
        c[i,j] = 20+round(Random.rand()*Random.rand(20:60),digits=2)
    end 
end

# fixed cost of each machine
h = [30*length(M) for i in M]

# duration of each job (for question (c))
d = [Random.rand(1:10)*Random.rand()+10 for j in N];

In [9]:
using JuMP, GLPK

# Initialize the model
model = Model(GLPK.Optimizer)

# Decision variables
@variable(model, x[M, N], Bin)  # 1 if job j is assigned to machine i
@variable(model, y[M], Bin)     # 1 if machine i is operated

# Objective: Minimize total cost
@objective(model, Min, sum(c[i, j] * x[i, j] for i in M, j in N) + sum(h[i] * y[i] for i in M))

# Constraint: Each job must be assigned to exactly one machine
@constraint(model, [j in N], sum(x[i, j] for i in M) == 1)


# Constraint: Total work on each machine must not exceed its capacity
@constraint(model, [i in M], sum(a[i, j] * x[i, j] for j in N) <= b[i] * y[i])

# Solve the model
optimize!(model)

# Get the results
optimal_value = objective_value(model)
job_assignment = value.(x)
machines_used = value.(y)

println("Objective value (Total Cost): ", optimal_value)
for i in M
    if machines_used[i] == 1
        println("Machine $i is operated with jobs: ", [j for j in N if job_assignment[i, j] > 0.5])
    else
        println("Machine $i is not operated")
    end
end

solve_duration = @elapsed optimize!(model)
println("Time taken to solve (seconds): ", solve_duration)

Objective value (Total Cost): 3780.9100000000003
Machine 1 is not operated
Machine 2 is not operated
Machine 3 is not operated
Machine 4 is not operated
Machine 5 is not operated
Machine 6 is operated with jobs: [4, 7, 9, 10, 11, 13, 14, 15, 17, 18, 20, 21, 22, 23, 27, 28, 29, 31, 33, 37, 45, 46, 48, 50, 51, 52, 53, 56, 58, 60]
Machine 7 is not operated
Machine 8 is not operated
Machine 9 is not operated
Machine 10 is not operated
Machine 11 is not operated
Machine 12 is not operated
Machine 13 is not operated
Machine 14 is not operated
Machine 15 is not operated
Machine 16 is not operated
Machine 17 is not operated
Machine 18 is not operated
Machine 19 is not operated
Machine 20 is not operated
Machine 21 is not operated
Machine 22 is not operated
Machine 23 is not operated
Machine 24 is not operated
Machine 25 is not operated
Machine 26 is not operated
Machine 27 is not operated
Machine 28 is operated with jobs: [1, 2, 3, 5, 6, 8, 12, 16, 19, 24, 25, 26, 30, 32, 34, 35, 36, 38, 39, 4

## (b)

In [3]:
using JuMP, GLPK

# Define the model using the GLPK optimizer
model = Model(GLPK.Optimizer)

# Define decision variables
@variable(model, x[1:length(M), 1:length(N)], Bin)  # Binary variable for job assignment to machines
@variable(model, y[1:length(M)], Bin)               # Binary variable to indicate if a machine is operated

# Objective function: Minimize total cost
@objective(model, Min, sum(c[i,j] * x[i,j] for i in 1:length(M), j in 1:length(N)) + sum(h[i] * y[i] for i in 1:length(M)))

# Constraint: Each job must be assigned to exactly one machine
@constraint(model, [j in 1:length(N)], sum(x[i,j] for i in 1:length(M)) == 1)

# Constraint: Total work on each machine must not exceed its capacity
@constraint(model, [i in 1:length(M)], sum(a[i,j] * x[i,j] for j in 1:length(N)) <= b[i] * y[i])

# LOGICAL CONDITION
# Binary variables to count active machines in each segment
@variable(model, z1, Bin)  # Indicator for at least 3 of the first 10 machines being active
@variable(model, z2, Bin)  # Indicator for at least 3 of the second 10 machines being active
@variable(model, z12, Bin) # Product of z1 and z2 (linearized)

# Ensure at least 3 of the first 10 machines are operated if z1 is active
@constraint(model, sum(y[i] for i in 1:10) >= 3 * z1)

# Ensure at least 3 of the second 10 machines are operated if z2 is active
@constraint(model, sum(y[i] for i in 11:20) >= 3 * z2)

# Linearization of z1 * z2
@constraint(model, z12 <= z1)
@constraint(model, z12 <= z2)
@constraint(model, z12 >= z1 + z2 - 1)

# If both z1 and z2 are active, operate at most 2 machines in the last 10 machines
@constraint(model, sum(y[i] for i in 21:30) <= 2 + 8 * (1 - z12))

# Optimize the model
optimize!(model)

# Retrieve the results
optimal_value_b = objective_value(model)
job_assignment_b = value.(x)
machines_used_b = value.(y)
solve_duration_b = @elapsed optimize!(model)

# Print the results
println("Objective value (Total Cost) for Part (b): ", optimal_value_b)
for i in 1:length(M)
    if machines_used_b[i] == 1
        println("Machine $i is operated with jobs: ", [j for j in 1:length(N) if job_assignment_b[i, j] > 0.5])
    else
        println("Machine $i is not operated")
    end
end

println("Time taken to solve Part (b) (seconds): ", solve_duration_b)

Objective value (Total Cost) for Part (b): 3780.9100000000003
Machine 1 is not operated
Machine 2 is not operated
Machine 3 is not operated
Machine 4 is not operated
Machine 5 is not operated
Machine 6 is operated with jobs: [4, 7, 9, 10, 11, 13, 14, 15, 17, 18, 20, 21, 22, 23, 27, 28, 29, 31, 33, 37, 45, 46, 48, 50, 51, 52, 53, 56, 58, 60]
Machine 7 is not operated
Machine 8 is not operated
Machine 9 is not operated
Machine 10 is not operated
Machine 11 is not operated
Machine 12 is not operated
Machine 13 is not operated
Machine 14 is not operated
Machine 15 is not operated
Machine 16 is not operated
Machine 17 is not operated
Machine 18 is not operated
Machine 19 is not operated
Machine 20 is not operated
Machine 21 is not operated
Machine 22 is not operated
Machine 23 is not operated
Machine 24 is not operated
Machine 25 is not operated
Machine 26 is not operated
Machine 27 is not operated
Machine 28 is operated with jobs: [1, 2, 3, 5, 6, 8, 12, 16, 19, 24, 25, 26, 30, 32, 34, 35, 

## (c)

In [10]:
using JuMP
using GLPK

M = 15
N = 15

# Model creation
model = Model(GLPK.Optimizer)

# Decision variables
@variable(model, x[1:M, 1:N], Bin)  # 1 if job j is assigned to machine i
@variable(model, y[1:M], Bin)       # 1 if machine i is used
@variable(model, makespan)          # Makespan to be minimized

# Objective: Minimize the makespan
@objective(model, Min, makespan)

# Each job must be assigned to exactly one machine
@constraint(model, [j in 1:N], sum(x[i,j] for i in 1:M) == 1)

# Total work on each machine must not exceed its capacity
@constraint(model, [i in 1:M], sum(a[i,j] * x[i,j] for j in 1:N) <= b[i])

# Makespan constraint: Time spent on any machine cannot exceed makespan
@constraint(model, [i in 1:M], makespan >= sum(d[j] * x[i,j] for j in 1:N))

# Use no more than half of the machines (15 out of 30)
@constraint(model, sum(y[i] for i in 1:M) <= 15)

# Solve the model
optimize!(model)

# Retrieve and print results
println("Optimal Makespan: ", objective_value(model))
for i in 1:M
    if value(y[i]) == 1
        println("Machine $i is used with jobs: ", [j for j in 1:N if value(x[i,j]) > 0.5])
    else
        println("Machine $i is not used")
    end
end

LoadError: InterruptException:

## (d)

### Using HiGHS

In [4]:
using JuMP, HiGHS

# Part (a): Least Cost Assignment with HiGHS
model_highs_a = Model(HiGHS.Optimizer)

@variable(model_highs_a, x[1:length(M), 1:length(N)], Bin)
@variable(model_highs_a, y[1:length(M)], Bin)

@objective(model_highs_a, Min, sum(c[i,j] * x[i,j] for i in 1:length(M), j in 1:length(N)) + sum(h[i] * y[i] for i in 1:length(M)))

@constraint(model_highs_a, [j in 1:length(N)], sum(x[i,j] for i in 1:length(M)) == 1)
@constraint(model_highs_a, [i in 1:length(M)], sum(a[i,j] * x[i,j] for j in 1:length(N)) <= b[i] * y[i])

# Solve the model
optimize!(model_highs_a)

# Retrieve and print the results
optimal_value_a = objective_value(model_highs_a)
job_assignment_a = value.(x)
machines_used_a = value.(y)

println("Objective value (Total Cost) for Part (a) with HiGHS: ", optimal_value_a)
for i in M
    if machines_used_a[i] == 1
        println("Machine $i is operated with jobs: ", [j for j in N if job_assignment_a[i, j] > 0.5])
    else
        println("Machine $i is not operated")
    end
end

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+02]
  Cost   [2e+01, 9e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
90 rows, 1830 cols, 3630 nonzeros  0s
90 rows, 1830 cols, 3630 nonzeros  0s
Objective function is integral with scale 100

Solving MIP model with:
   90 rows
   1830 cols (1830 binary, 0 integer, 0 implied int., 0 continuous)
   3630 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   0               23041.32         100.00%        0      0      0         0     0.0s
 R       0       0         0   0.00%   267

In [5]:
# Part (b): Logical Constraints with HiGHS
model_highs_b = Model(HiGHS.Optimizer)

@variable(model_highs_b, x[1:length(M), 1:length(N)], Bin)
@variable(model_highs_b, y[1:length(M)], Bin)

@objective(model_highs_b, Min, sum(c[i,j] * x[i,j] for i in 1:length(M), j in 1:length(N)) + sum(h[i] * y[i] for i in 1:length(M)))

@constraint(model_highs_b, [j in 1:length(N)], sum(x[i,j] for i in 1:length(M)) == 1)
@constraint(model_highs_b, [i in 1:length(M)], sum(a[i,j] * x[i,j] for j in 1:length(N)) <= b[i] * y[i])

# Logical condition
@variable(model_highs_b, z1, Bin)
@variable(model_highs_b, z2, Bin)
@variable(model_highs_b, z12, Bin)

@constraint(model_highs_b, sum(y[i] for i in 1:10) >= 3 * z1)
@constraint(model_highs_b, sum(y[i] for i in 11:20) >= 3 * z2)
@constraint(model_highs_b, z12 <= z1)
@constraint(model_highs_b, z12 <= z2)
@constraint(model_highs_b, z12 >= z1 + z2 - 1)
@constraint(model_highs_b, sum(y[i] for i in 21:30) <= 2 + 8 * (1 - z12))

# Solve the model
optimize!(model_highs_b)

# Retrieve and print the results
optimal_value_b = objective_value(model_highs_b)
job_assignment_b = value.(x)
machines_used_b = value.(y)

println("Objective value (Total Cost) for Part (b) with HiGHS: ", optimal_value_b)
for i in M
    if machines_used_b[i] == 1
        println("Machine $i is operated with jobs: ", [j for j in N if job_assignment_b[i, j] > 0.5])
    else
        println("Machine $i is not operated")
    end
end

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+02]
  Cost   [2e+01, 9e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+01]
Presolving model
96 rows, 1833 cols, 3670 nonzeros  0s
90 rows, 1830 cols, 3630 nonzeros  0s
Objective function is integral with scale 100

Solving MIP model with:
   90 rows
   1830 cols (1830 binary, 0 integer, 0 implied int., 0 continuous)
   3630 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   0               23041.32         100.00%        0      0      0         0     0.0s
 R       0       0         0   0.00%   267

In [ ]:
# Part (c): Minimizing Makespan with HiGHS
model_highs_c = Model(HiGHS.Optimizer)

@variable(model_highs_c, x[M, N], Bin)
@variable(model_highs_c, y[M], Bin)
@variable(model_highs_c, makespan)

@objective(model_highs_c, Min, makespan)

@constraint(model_highs_c, [j in N], sum(x[i,j] for i in M) == 1)
@constraint(model_highs_c, [i in M], sum(a[i,j] * x[i,j] for j in N) <= b[i] * y[i])
@constraint(model_highs_c, [i in M, j in N], x[i,j] <= y[i])

for i in M
    @constraint(model_highs_c, makespan >= sum(d[j] * x[i,j] for j in N))
end

@constraint(model_highs_c, sum(y[i] for i in M) <= 15)

# Solve the model
optimize!(model_highs_c)

# Retrieve and print the results
optimal_makespan = objective_value(model_highs_c)
job_assignment_c = value.(x)
machines_used_c = value.(y)

println("Optimal Makespan for Part (c) with HiGHS: ", optimal_makespan)
for i in M
    if machines_used_c[i] == 1
        println("Machine $i is operated with jobs: ", [j for j in N if job_assignment_c[i, j] > 0.5])
    else
        println("Machine $i is not operated")
    end
end

# Problem 2

In [7]:
using JuMP
using GLPK

# Number of batches
n = 10

# Time matrix representing cleaning times between batches
time_matrix = [
    0 11 7 13 11 12 4 9 7 11;
    5 0 13 15 15 6 8 10 9 8;
    13 15 0 23 11 11 16 18 5 7;
    9 13 5 0 8 10 12 14 5 3;
    3 7 7 7 0 9 10 11 12 13;
    10 6 3 4 14 0 8 5 11 12;
    4 6 7 3 13 7 0 10 4 6;
    7 8 9 9 12 11 10 0 10 9;
    9 9 14 8 4 9 6 10 0 12;
    11 17 11 6 10 4 7 9 11 0;
]

# Mixing times for each batch
mixing_times = [40, 35, 45, 32, 50, 42, 44, 30, 33, 55]

# Create the model
model = Model(GLPK.Optimizer)

# Decision Variables
@variable(model, x[1:n, 1:n], Bin)    # x[i,j] = 1 if we go from batch i to batch j, 0 otherwise
@variable(model, u[2:n] >= 0)         # Subtour elimination variables (MTZ formulation)

# Objective: Minimize the total time (mixing + cleaning)
@objective(model, Min, 
    sum(time_matrix[i,j] * x[i,j] for i in 1:n, j in 1:n) +  # Cleaning time
    sum(mixing_times[i] * sum(x[i,j] for j in 1:n) for i in 1:n)  # Mixing time
)

# Constraints: Ensure each batch is entered and left exactly once
@constraint(model, [i=1:n], sum(x[i,j] for j in 1:n if i != j) == 1)
@constraint(model, [j=1:n], sum(x[i,j] for i in 1:n if i != j) == 1)

# Subtour elimination constraints (MTZ formulation)
@constraint(model, [i=2:n, j=2:n], u[i] - u[j] + n*x[i,j] <= n-1)

# Solve the model
optimize!(model)

# Extract the optimal order of paint batches
optimal_order = []
current_batch = 1
push!(optimal_order, current_batch)

while length(optimal_order) < n
    for j in 1:n
        if value(x[current_batch, j]) > 0.5 && !(j in optimal_order)
            push!(optimal_order, j)
            current_batch = j
            break
        end
    end
end

# Print the optimal order and total time
println("Optimal Order of Paint Batches: ", optimal_order)
println("Total Time (Mixing + Cleaning): ", objective_value(model))

Optimal Order of Paint Batches: Any[1, 7, 4, 10, 8, 2, 6, 3, 9, 5]
Total Time (Mixing + Cleaning): 454.0
